In [ ]:
import time as T 
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.datasets as datsets
import torchvision.transforms as transforms

from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
import pickle as P
import os 
#Set reference working directory to be parent folder 
#import os 
#os.chdir('..')

In [12]:
#print(os.getcwd())

In [13]:
#Pickle based upload and load python file helper function  

def uploadObject(Object, filename, nosilence=True):
    t0 = T.time()
    i=1
    while 1==1:
        try:
            filehandler = open(filename, 'wb') 
            
            P.dump(Object, filehandler)
            loadObject(filename)
            if nosilence:
                print('----------------FINISHED SAVING----------------')
            break
        except EOFError:
            if nosilence:
                print(f'Attempt #{i} failed')
            i+=1
            if i == 11:
                if nosilence:
                    print('Failed to Save')
                break
            
        
    t1 = T.time() - t0
    if nosilence:
        print(f'Time: {t1-t0:.5f} s\n')
    
def loadObject(filename, nosilence=True):
    t0 = T.time()
    infile = open(filename,'rb')
    Object = P.load(infile)
    infile.close()
    t1 = T.time()
    if nosilence:
        print(f'Time: {t1-t0:.5f} s\n')
    return Object

In [14]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [15]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [16]:
#Split the train and test indicies 
def split_indices(n,val_pct):
    #determine size of validation set
    n_val = int(n*val_pct)
    #create random permutation of 0 to n-1
    idxs = np.random.permutation(n)
    #Pick first n_val random indices as validation set
    return idxs[n_val:], idxs[:n_val]

In [17]:
c = 1 
if str(os.getcwd())[-16:] != "AI II FINAL CODE":
    #Go to the parent folder AI II FINAL CODE as main path 
    try: 
        import os
        os.chdir('../../AI II FINAL CODE/')
    except:
        pass 

In [ ]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:

if c > 0: 
    transform=transforms.Compose([
    transforms.ToTensor()])
    #AddGaussianNoise(0., 1.) ])
    
    #Import dataset from data folder in parent folder 
    dataset = datsets.FashionMNIST(root='data/', download=False, transform=transform)

In [21]:
len(dataset)

60000

In [22]:
if c > 1: 
    #Show image of one example 
    image, label = dataset[0]
    print('image.shape:', image.shape)
    plt.imshow(image.permute(1, 2, 0), cmap='gray')
    print('Label:', label)

In [23]:
if c > 0:
    #Make 40% of the data testing data 
    train_indices, val_indices = split_indices(len(dataset), 0.4)

    if c > 1: 
        print(len(train_indices))
        print(len(val_indices))
        print("Random Validation Samples:", val_indices[:5])

In [ ]:
train_indices

(36000,)

In [ ]:
if c > 0: 
    #Create the training and testing batches 
    batch_size=512

    #Training sampler and data loader
    train_sampler = SubsetRandomSampler(train_indices)
    train_loader = DataLoader(dataset, 
                            batch_size, sampler = train_sampler)

    #Validation sampler and data loader
    val_sampler = SubsetRandomSampler(val_indices)
    val_loader = DataLoader(dataset, 
                            batch_size, sampler = val_sampler)

In [ ]:
if c > 1: 
    #Show the images of each batch 
    for images, _ in train_loader:
        print('images.shape:', images.shape)
        plt.figure(figsize=(16,8))
        plt.axis('off')
        plt.imshow(make_grid(images, nrow=16).permute((1, 2, 0)))
        break

In [ ]:
if c > 0: 
    #USe cuda when avilable 
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(f"Using device: {device}")

In [ ]:
if c > 1: 
    for images, labels in train_loader:
        print(images.shape)
        images = to_device(images, device)
        print(images.device)
        break

In [ ]:
if c > 0: 
    #Load the training and testing data 
    train_loader = DeviceDataLoader(train_loader, device)
    val_loader = DeviceDataLoader(val_loader, device)